# Elektronska fiskalizacija

**Primjeri kod-a u Python-u**

Za primjere u prilogu je neophodno imati `signxml`, `pycrypto` pa to valja prethodno instalirati (`!pip install signxml pycrypto`). Osim toga, podrazumijeva se posjedovanje test PFX sertifikata (_CoreitPecatSoft.pfx_) u lokalnom folderu.

**sstevan**: Možete pokrenuti i `pip install -r requirements.txt ` da se instaliraju svi neophodni paketi.


In [1]:
pfx_cert = 'CoreitPecatSoft.pfx'
pfx_password = b'123456'

### Priprema sertifikata

Ekstrakcija privatnog kljuca i sertifikata iz PFX kontejnera. Za taj dio koristim kod dostupan na sljedecem [linku](https://www.jhanley.com/google-cloud-extracting-private-key-from-service-account-p12-credentials/) 

In [2]:
import OpenSSL.crypto
import os

###########################################################    
# Version 1.00
# Date Created: 2018-12-21
# Last Update:  2018-12-21
# https://www.jhanley.com
# Copyright (c) 2018, John J. Hanley
# Author: John Hanley
###########################################################

# Convert a Google P12 (PFX) service account into private key and certificate.
# Convert an SSL Certifcate (PFX) into private key, certificate and CAs.

def write_CAs(filename, p12):
    # Write the Certificate Authorities, if any, to filename

    if os.path.exists(filename):
        os.remove(filename)

    ca = p12.get_ca_certificates()

    if ca is None:
        return

    print('Creating Certificate CA File:', filename)

    with open(filename, 'wb') as f:
        for cert in ca:
            f.write(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_PEM, cert))

def pfx_to_pem(pfx_path, pfx_password, pkey_path, pem_path, pem_ca_path):
    '''
    Decrypt the P12 (PFX) file and create a private key file and certificate file.

    Input:
        pfx_path    INPUT: This is the Google P12 file or SSL PFX certificate file
        pfx_password    INPUT: Password used to protect P12 (PFX)
        pkey_path   INPUT: File name to write the Private Key to
        pem_path    INPUT: File name to write the Certificate to
        pem_ca_path INPUT: File name to write the Certificate Authorities to
    '''

    print('Opening:', pfx_path)
    with open(pfx_path, 'rb') as f_pfx:
        pfx = f_pfx.read()

    print('Loading P12 (PFX) contents:')
    p12 = OpenSSL.crypto.load_pkcs12(pfx, pfx_password)

    print('Creating Private Key File:', pkey_path)
    with open(pkey_path, 'wb') as f:
        # Write Private Key
        f.write(OpenSSL.crypto.dump_privatekey(OpenSSL.crypto.FILETYPE_PEM, p12.get_privatekey()))

    print('Creating Certificate File:', pem_path)
    with open(pem_path, 'wb') as f:
        # Write Certificate
        f.write(OpenSSL.crypto.dump_certificate(OpenSSL.crypto.FILETYPE_PEM, p12.get_certificate()))

    # Google P12 does not have certifiate authorities but SSL PFX certificates do
    write_CAs(pem_ca_path, p12)


# Start here
pfx_to_pem(
    pfx_cert,         # Google Service Account P12 file
    pfx_password,     # P12 file password
    'coreit.key',      # Filename to write private key
    'coreit_cert.pem', # Filename to write certificate
    'coreit_ca.pem')   # Filename to write CAs if present

Opening: CoreitPecatSoft.pfx
Loading P12 (PFX) contents:
Creating Private Key File: coreit.key
Creating Certificate File: coreit_cert.pem
Creating Certificate CA File: coreit_ca.pem


#### Odredjivanje datuma i vremena

In [3]:
from datetime import datetime
import pytz

current_time = datetime.now(pytz.timezone('Europe/Podgorica')).strftime('%Y-%m-%dT%H:%M:%S%z')
current_time = "{0}:{1}".format(current_time[:-2], current_time[-2:])

#### Generisanje UUID-a

In [4]:
import uuid
fresh_uuid = str(uuid.uuid4())

### Generisanje IKOF-a

In [5]:
import hashlib
import sys
import time

from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5 as Cipher_PKCS1_v1_5
from Crypto.Signature import PKCS1_v1_5
from base64 import b64decode, b64encode
from Crypto.Hash import SHA256

# sstevan: Python 3.8 nema time.clock() pa pycrypto pravi probleme
if sys.version_info >= (3, 8):
    time.clock = time.process_time


def generate_iis(data, private_key_path):
    message = bytes(data, 'utf8')
    digest = SHA256.new()
    digest.update(message)    
    private_key = False
    with open(private_key_path, "r") as key_file:
        private_key = RSA.importKey(key_file.read())
    signer = PKCS1_v1_5.new(private_key)
    sig = signer.sign(digest)
    iic = hashlib.md5(sig).digest()
    
    return (sig.hex(), iic.hex())

#### Primjer:

In [6]:
(iic, ikof) = generate_iis(
    "{0}|{1}|{2}|{3}|{4}|{5}|{6}".format(
        "12345678",
        current_time,
        "9952",
        "bb123bb123",
        "cc123cc123",
        "ss123ss123",
        "99.01"
    ),
    'coreit.key'
)
print("IIC:\n", iic, "\nIKOF:\n", ikof)

IIC:
 69ce3e82c8bc7faff31065d060ff4b2ed1aba803b7358774c897c3dab4df5ecf7c47e3a301e7ba9fd85b43be278916f139c40e8ee5cf9990fdb3e8f4fca50bf43d329253eaad1b7419e72b1a6cf9fab12ec2b164add1579b1ee0876b6e906c623206f7da16c239b50f6a1f64954ce1aafdc14a3ece321eebab4f4cf797a03c7800f880c57ae39a4005e4bb877ff59498266cba359fcd6dbf1e46ebabc1e12f58540f35a3daca471080022d58d1d0503196831993be7a9af549e31f43053629dbe295fcbddafee11ea9278a06882933c2eacb59738bcab83fb26fb98699741993130a5fa5a50063f12df1374f4127d73ab65ddda35c3d4724ed8a2c3ef51134ff 
IKOF:
 369f42b3f5121ab28ca1da1c1993092d


### Priprema i potpisivanje XML-a

#### Priprema XML objekta i parsiranje XML sadrzaja

In [7]:
xml = '<?xml version="1.0"?><RegisterInvoiceRequest xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" Id="Request" Version="1" xmlns="https://efi.tax.gov.me/fs/schema"><Header UUID="{uid}" SendDateTime="{now}" /><Invoice TypeOfInv="CASH" IsSimplifiedInv="false" IssueDateTime="{now}" InvNum="10/2020/nt360fe389" InvOrdNum="10" TCRCode="nt360fe389" IsIssuerInVAT="true" TotPriceWoVAT="10.00" TotPrice="12.10" OperatorCode="oo123oo123" BusinUnitCode="xx123xx123" SoftCode="ss123ss123" IIC="{ikof}" IICSignature="{iic}" IsReverseCharge="false"><PayMethods><PayMethod Type="BANKNOTE" Amt="12.10" /></PayMethods><Seller IDType="TIN" IDNum="12345678" Name="Test d.o.o" /><Items><I N="Test artikal" C="1234234" U="kom" Q="10" UPB="1.21" UPA="1.21" PB="10.00" VR="21.00" VA="2.10" PA="12.10" /></Items></Invoice></RegisterInvoiceRequest>'

from lxml import etree
xml_obj = etree.fromstring(xml.format(uid=fresh_uuid, now=current_time, ikof=ikof, iic=iic))

#### Potpisivanje

In [8]:
from lxml import etree
from signxml import XMLSigner, XMLVerifier, methods

cert = open("coreit_cert.pem", 'rb').read()
key = open("coreit.key", 'rb').read()

signer = XMLSigner(
    method = methods.enveloped,
    signature_algorithm = 'rsa-sha256',
    digest_algorithm = 'sha256',
    c14n_algorithm = 'http://www.w3.org/2001/10/xml-exc-c14n#'
)

# Uklanjanje suvisnih DS namespace prefix-a prema preporuci sa: 
# https://github.com/XML-Security/signxml/issues/30#issuecomment-149618525
ns = {}
ns[None] = signer.namespaces['ds']
signer.namespaces = ns

# Konacno potpisivanje
signed_xml = signer.sign(
    xml_obj, 
    key=key,
    cert=cert
)

#### Rezultat

In [9]:
print(etree.tostring(signed_xml, encoding='utf8').decode('utf8'))

<RegisterInvoiceRequest xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="https://efi.tax.gov.me/fs/schema" Id="Request" Version="1"><Header UUID="652fc797-a996-4cd4-83b1-3a50e201dc9c" SendDateTime="2020-11-03T17:42:14+01:00"/><Invoice TypeOfInv="CASH" IsSimplifiedInv="false" IssueDateTime="2020-11-03T17:42:14+01:00" InvNum="10/2020/nt360fe389" InvOrdNum="10" TCRCode="nt360fe389" IsIssuerInVAT="true" TotPriceWoVAT="10.00" TotPrice="12.10" OperatorCode="oo123oo123" BusinUnitCode="xx123xx123" SoftCode="ss123ss123" IIC="369f42b3f5121ab28ca1da1c1993092d" IICSignature="69ce3e82c8bc7faff31065d060ff4b2ed1aba803b7358774c897c3dab4df5ecf7c47e3a301e7ba9fd85b43be278916f139c40e8ee5cf9990fdb3e8f4fca50bf43d329253eaad1b7419e72b1a6cf9fab12ec2b164add1579b1ee0876b6e906c623206f7da16c239b50f6a1f64954ce1aafdc14a3ece321eebab4f4cf797a03c7800f880c57ae39a4005e4bb877ff59498266cba359fcd6dbf1e46ebabc1e12f58540f35a3daca471080022d58d1d0503196831993be7a9af549e31

### Formiranje SOAP zahtjeva ([@sstevan](https://github.com/sstevan))

In [10]:
# sstevan: Formiranje i slanje SOAP zahtjeva

soap_ns = 'http://schemas.xmlsoap.org/soap/envelope/'
ns_map = {'soapenv': soap_ns}

envelope = etree.Element(etree.QName(soap_ns, 'Envelope'), nsmap=ns_map)
header = etree.SubElement(envelope, etree.QName(soap_ns, 'Header'), nsmap=ns_map)
body = etree.SubElement(envelope, etree.QName(soap_ns, 'Body'), nsmap=ns_map)

# sstevan: Dodavanje prethodno potpisano zahtjeva RegisterInvoiceRequest
reg_invoice_req = body.append(signed_xml)
soap_request = etree.tostring(envelope, encoding='utf8')

print(soap_request.decode('utf8'))

<soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/"><soapenv:Header/><soapenv:Body><RegisterInvoiceRequest xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="https://efi.tax.gov.me/fs/schema" Id="Request" Version="1"><Header UUID="652fc797-a996-4cd4-83b1-3a50e201dc9c" SendDateTime="2020-11-03T17:42:14+01:00"/><Invoice TypeOfInv="CASH" IsSimplifiedInv="false" IssueDateTime="2020-11-03T17:42:14+01:00" InvNum="10/2020/nt360fe389" InvOrdNum="10" TCRCode="nt360fe389" IsIssuerInVAT="true" TotPriceWoVAT="10.00" TotPrice="12.10" OperatorCode="oo123oo123" BusinUnitCode="xx123xx123" SoftCode="ss123ss123" IIC="369f42b3f5121ab28ca1da1c1993092d" IICSignature="69ce3e82c8bc7faff31065d060ff4b2ed1aba803b7358774c897c3dab4df5ecf7c47e3a301e7ba9fd85b43be278916f139c40e8ee5cf9990fdb3e8f4fca50bf43d329253eaad1b7419e72b1a6cf9fab12ec2b164add1579b1ee0876b6e906c623206f7da16c239b50f6a1f64954ce1aafdc14a3ece321eebab4f4cf797a03c7800f880c57a

### Slanje SOAP zahtjeva ([@sstevan](https://github.com/sstevan))

In [11]:
# Slanje SOAP zahtjeva
import requests
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
efi_url = 'https://efitest.tax.gov.me:443/fs-v1'

headers = {'content-type': 'text/xml'}

# Da izbjegnemo 'SSL: CERTIFICATE_VERIFY_FAILED' gresku dodajemo verify=False
response = requests.post(efi_url, data=soap_request,headers=headers, verify=False)
xml_response = etree.fromstring(response.content)

print(etree.tostring(xml_response, encoding='utf8', pretty_print=1).decode('utf8'))

<env:Envelope xmlns:env="http://schemas.xmlsoap.org/soap/envelope/">
  <env:Header/>
  <env:Body>
    <RegisterInvoiceResponse xmlns="https://efi.tax.gov.me/fs/schema" xmlns:ns0="http://www.w3.org/2000/09/xmldsig#" Id="Response" Version="1">
      <Header xmlns="https://efi.tax.gov.me/fs/schema" UUID="86b5ac73-0996-420a-99fe-667e02fa7c80" RequestUUID="652fc797-a996-4cd4-83b1-3a50e201dc9c" SendDateTime="2020-11-03T17:42:14+01:00"/>
      <FIC xmlns="https://efi.tax.gov.me/fs/schema">bf994091-fd6c-43b5-987d-e99a874a77b3</FIC>
      <Signature xmlns="http://www.w3.org/2000/09/xmldsig#">
        <SignedInfo>
          <CanonicalizationMethod Algorithm="http://www.w3.org/2001/10/xml-exc-c14n#"/>
          <SignatureMethod Algorithm="http://www.w3.org/2001/04/xmldsig-more#rsa-sha256"/>
          <Reference URI="#Response">
            <Transforms>
              <Transform Algorithm="http://www.w3.org/2000/09/xmldsig#enveloped-signature"/>
              <Transform Algorithm="http://www.w3.org

### I, najzad, FIC (JIKR) je

In [12]:
# fic = xml_response.find('.//FIC')
fic = xml_response.find('.//{https://efi.tax.gov.me/fs/schema}FIC')
fic.text

'bf994091-fd6c-43b5-987d-e99a874a77b3'